In [85]:
import numpy as np
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
# load the model
model = load_model('model_upper_40_n1.h5')
# load the mapping
mapping = load(open('mapping_upper.pkl', 'rb'))
#firsts = load(open('first_freqs.pkl', 'rb'))

In [86]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re

In [87]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed, n_chars):
    in_text = seed
    #np.random.choice(list(firsts.keys()), 1, list(firsts.values()))[0]
    print(in_text)
    len(in_text)
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        encoded.shape
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
            # append to input
        in_text += out_char
    return in_text

In [88]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# load text
#spenser = load_doc('data/spenser.txt')
#spenser = "\n\n" + spenser
#spenser = re.sub("\n\n(.*)\n\n", "\n\n", spenser)[2:]
shakes = load_doc('data/shakespeare.txt')
shakes = ''.join([i for i in shakes if not i.isdigit()])
#raw_text = spenser + shakes
raw_text = shakes
tokens = raw_text.split()
raw_text = ' '.join(tokens)

#organize into sequences of characters
length = 40
n = 5
sequences = list()
for i in range(length, len(raw_text), n):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
    #print('Total Sequences: %d' % len(sequences))

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()    
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)


#load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))


sequences = list()
for line in lines:
    #integer encode line
    encoded_seq = [mapping[char] for char in line]
    #store
    sequences.append(encoded_seq)
#vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)
X.shape

Vocabulary Size: 61


(18727, 40, 61)

In [89]:
temp_weights = [layer.get_weights() for layer in model.layers]


In [90]:
# test start of rhyme
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Lambda(lambda x: x / 1))
model.add(Dense(vocab_size, activation='softmax'))
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
model.layers[0].set_weights(temp_weights[0])

In [92]:
model.layers[2].set_weights(temp_weights[1])

In [93]:
print(generate_seq(model, mapping, 40, "", 2000))


 ty thrue show, To dwe mourn found it in the strength wrong, And so it were your self will be thy deeds, The part of thy self thy self well doth lie, Mad shar when it doth that thou dost best To be a tit it seem the mone thee him, And the present of me to good with thee here, And therefore to the world which in thy brow, For where thou art my beloved of my mine, Thou best is my self to my shadow me, I love thee alone deceiving him thy sweet, To should that beauty of thy self aming, And you and therefore doth power to grans, And be a summen doth lived a mowled stand Upon thy self answeetts do not so, Then my love that thou thought thee is my heart, And therefore with the world that thou despive, Though no conttim) me than his beauty lees, The bounteous shame delight when thou art more with thee, Who that thou thy self doth be ten sings Poor ranong my verse is thy should to me, The beauty of thy self amsevoly show, Thing enjoy the thought of my sing of me, To me that I importic this I c